In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import *

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms, models
from torchvision.datasets import ImageFolder, DatasetFolder

from transformers import ViTImageProcessor, ViTForImageClassification

In [16]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Linear(32 * 16 * 16, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [17]:
model = SimpleCNN()

In [18]:
class DataModule:
    def __init__(
        self,
        dataset_path,
        transform,
        batch_size,
        num_workers,
    ):
        print(dataset_path)

        # Create the training dataset
        self.dataset = ImageFolder(dataset_path, transform = transform)
        
        # Split the dataset in 5
        self.dataset_list = torch.utils.data.random_split(self.dataset, [0.7, 0.3])

        self.batch_size = batch_size
        self.num_workers = num_workers
    
    def val_dataloader_list(self):
        """ Returns the list containing a training DataLoader and a test Dataloader """
        val_loader_list = [DataLoader(
                            x,
                            batch_size = self.batch_size,
                            shuffle = False,
                            num_workers = self.num_workers,
                            ) for x in self.dataset_list]
        return val_loader_list

    def train_dataloader(self, i):
        """
        i: indicates which train set has to be used (i is 0, 1, 2, 3 or 4)

        Returns a DataLoader containing 80% of the training set, and the pseudo-labelled images
        """
        self.concat = torch.utils.data.ConcatDataset([self.dataset_list[j] for j in range(5) if j != i]
                                                     + [self.pseudo_labelled_dataset])

        train_loader = DataLoader(
                        self.concat,
                        batch_size = self.batch_size,
                        shuffle = True,
                        num_workers = self.num_workers
                        )
        
        return train_loader

In [19]:
batch_size = 16
lr = 1e-3
epoch = 10

In [20]:
dataset_path = "/Users/pierreaguie/X_AI_Hackathon/data/images/"
transform = transforms.ToTensor()

In [21]:
data = DataModule(dataset_path, transform, batch_size, 4)

/Users/pierreaguie/X_AI_Hackathon/data/images/


In [22]:
sets = data.val_dataloader_list()
training_set, test_set = sets[0], sets[1]

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [24]:
def success_rate(model,dataloader):
    nb = 0
    success = 0
    results = []
    for i, (input, target) in enumerate(dataloader):
        input, target = input.to(device), target.to(device)
        with torch.no_grad():
          y_pred = model(input)
        pred = torch.argmax(y_pred, dim = -1)
        for i in range(int(pred.shape[0])):
            if pred[i] == target[i]:
                success += 1
            nb += 1
    return success / nb

In [25]:
def train(model, device, dataloader, epoch, rate = 1e-4):
    
    model.to(device)
    model.train()

    optimizer = torch.optim.SGD(model.parameters(), lr=rate)

    loss_fn = nn.CrossEntropyLoss()
    train_losses = []
    accuracy = []
    for t in range(epoch):
        for i, (input_data, target) in enumerate(dataloader):

            input_data, target = input_data.to(device), target.to(device)

            #on calcule la prediction 
            y_pred = model(input_data)
            loss = loss_fn(y_pred, target)
            optimizer.zero_grad()
            #back propagation
            loss.backward()
            optimizer.step()
        loss_val = loss.detach().item()
        print("Epoch : ", t , " | ", loss_val)
        train_losses.append(loss_val)
        accuracy = success_rate(model, test_set)

    return train_losses, accuracy

In [26]:
train(model, device, training_set, epoch, rate = lr)

100%|██████████| 10/10 [07:31<00:00, 45.15s/it]


([0.7022380828857422,
  0.6992009282112122,
  0.6957888603210449,
  0.6924731731414795,
  0.6892526149749756,
  0.6861075758934021,
  0.6830282211303711,
  0.6800026297569275,
  0.6770260334014893,
  0.6740804314613342],
 0.6796875)